<a id="start"></a>
# **Embedded Convolutional Neural Network - Tutorial Prático**

Esta é a parte prática do tutorial de construção de um modelo de colorização de imagens classificadas. Nessa `Notebook`, você será guiado a construir código do modelo **ECNN** e também os códigos necessários para **treinar** o modelo, **testar** acurácia e **aplicar** o modelo para colorir novas imagens.

Esse tutorial possuirá pouca teoria, espera-se você já tenha lido a parte teórica nessecessária para entender o que será feito aqui presente em nosso [site](https://rafaeldbo.github.io/project-colorization/context). De qualquer maneira, as sessões necessárias para entendimento de cada etapa desse `Notebook` serão referênciadas para consulta na própria etapa. 

## **Sumário**

1. [Preparando as Imagens](#load_images)
2. [Construindo o Modelo](#building_model)

    2.1. [Criando a Estrutura do Modelo](#creating_layers)

    2.2. [Função "foward"](#foward)
    
    2.3. [Modelo Completo](#ECNN_model)
3. [Construindo a Rotina de Treinamento](#train_model)
4. [Construindo a Rotina de Testes](#test_model)
5. [Aplicando o Modelo](#deploy_model)

## **Instalando Dependências**
 
Antes de começarmos, caso não tenha as bibliotecas necessárias, rode a célula abaixo para instala-las ou instale utilizando o arquivo [requirements.txt](https://github.com/rafaeldbo/project-colorization/blob/main/requirements.txt) presente em nosso repositório. É recomendado o uso de um ambiente virtual. 

In [ ]:
# Só é necessário rodar esse script uma vez para instalar as dependências necessárias
!python -m pip install torch scikit-image matplotlib pandas numpy jupyter ipykernel

Caso possuia **GPU** com suporte ao `cuda` e deseja utiliza-la, verifique se ela já está disponivel utilizando o código abaixo. Caso não esteja, acesse esse [site](https://pytorch.org/get-started/locally/) e escolha a versão do `pytorch` com maior compatibilidade com sua GPU e a instale.

In [4]:
import torch

print(f"versão do torch: {torch.__version__}")
print("cuda disponivel!" if torch.cuda.is_available() else "cuda indisponível :(")

versão do torch: 2.5.1+cu124
cuda disponivel!


In [7]:
# Importando as bibliotecas necessárias
import torch
import pandas as pd

from torch import nn, Tensor,  cat, from_numpy, save, load
from torch.nn.functional import relu
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from skimage.io import imread
from skimage.color import rgb2lab

from os import path, listdir, mkdir

<a id="load_images"></a>
## **Preparando as Imagens**

A teoria referente a essa etapa está disponível na sessão [Entradas e Saídas do Modelo](https://rafaeldbo.github.io/project-colorization/inputs_outputs) do nosso site.

Um ponto importante do modelo que criaremos nesse tutorial é que, após ser treinado por um conjunto de imagens de determinadas dimensões ($largura \times altura$), ele só poderá ser aplicado em imagens destas mesmas dimensões. Nesse tutorial, utilizaremos as imagens do dataset [Image Colorization Dataset](https://www.kaggle.com/datasets/aayush9753/image-colorization-dataset) que possuem as dimensões $400 \times 400$, dessa forma, todas as imagens recebidas e "geradas" pelo modelo possuirão essas mesmas dimensões. Além disso, como se trata de um modelo que colore imagem caregorizadas, também precisaremos de um arquivos com as categorias de cada imagem. Felizmente nosso time já preparou o aquivo [categories.csv](https://alinsperedu-my.sharepoint.com/:x:/g/personal/rafaeldbo_al_insper_edu_br/ETV6ST4HWAFFhvtF-JJ5HjsB_v9Fe3QacOhVpd3ynIYiyA?e=2W7cAB) exatamente com essa informação. Coloque tanto as pastas de imagens do dataset quanto o arquivo de categorias em uma pasta `data`, para melhor organização.

Para preparar as imagens para a utilização pelo nosso modelo precisaremos criar um classe de **dataset** personalisada baseada na classe `Dataset` do `pytorch`. Ela deverá localizar todas a imagens que serão utilizadas pelo modelo e possuir uma função `__getitem__` que  caregará uma imagem de cada vez, fornecendo os dados da imagem e sua categoria. Essa estrutura será imoprtante para que, no futuro, o `DataLoader` seja capaz de utilizar essa classe **dataset** para carregar as imagens dos nossos **Batches** (caso não se lembre do que estamos falando, isso será retomado mais adiante). Para facilitar, já iremos fazer com que ela retorne separadamente o layer L e os layers AB.

O código desse dataset personalizado será:

In [9]:
class ImageDataset(Dataset):

    def __init__(self,
        images_path: str, # Caminho da pasta onde estão as imagens 
        categories_file: str, # Caminho do arquivo csv com as categorias das imagens
        size: int = -1, # Quantidade de imagens a serem carregadas, sendo -1 para todas
    ):
        # criando uma lista com os arquivos das imagens
        self.images_path = images_path
        images = listdir(images_path) # listando os arquivos da pasta
        size = size if size > 0 else len(images)
        self.images_files = images[:size]
        
        # criando um dicionário com as categorias das imagens
        df = pd.read_csv(path.join(categories_file), delimiter=';') # lendo o arquivo csv
        df['category'] = df['category'].fillna(0) # colocando a categoria 0 para as imagens sem categoria, caso existam
        self.categories = df.set_index('image')['category'].to_dict() # criando o dicionário

    # função que retorna o tamanho do dataset
    def __len__(self):
        return len(self.images_files)

    def __getitem__(self, 
        index: int, # Índice da imagem a ser carregada
    ) -> tuple[Tensor, Tensor, int]:
        
            # lendo a imagem
            img_file = self.images_files[index]
            img_path = path.join(self.images_path, img_file)
            img = imread(img_path) 

            # converte a imagem de RGB para LAB
            LAB_img = from_numpy(rgb2lab(img)) 
            LAB_img = LAB_img.permute(2, 0, 1) 

            # separa os layers
            gray_layer = LAB_img[0, :, :].unsqueeze(0) 
            color_layers = LAB_img[1:, :, :] 
            
            # retornando o layer L, os layers AB e a categoria da imagem
            return gray_layer, color_layers, self.categories[img_file]